In [87]:
# import Libraries

import openai
import langchain
import pinecone 
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.llms import OpenAI
import time

In [88]:
from dotenv import load_dotenv
load_dotenv()

True

In [89]:
import os

In [90]:
## Lets Read the document
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [92]:
doc = read_doc("./documents/")
doc

[Document(page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023', metadata={'source': 'documents/budget_speech.pdf', 'page': 0}),
 Document(page_content='', metadata={'source': 'documents/budget_speech.pdf', 'page': 1}),
 Document(page_content='CONTENTS \nPART-A \n Page No.  \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal  – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector  \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management 24 \nPART B  \n  \nIndirect Taxes  27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metal

In [93]:
## Divide the docs into chunks
### https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [94]:
documents = chunk_data(docs=doc)
len(documents)

141

In [113]:
type(documents[0])

langchain_core.documents.base.Document

In [20]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1173e4fd0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x117a87050>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None)

In [27]:
vectors=embeddings.embed_query("Who are you?")
len(vectors)

1536

In [28]:
vectors

[0.018032708940650094,
 -0.013798720984716732,
 -0.006982255185517665,
 -0.01469142998436532,
 -0.03387190737275715,
 0.01360742639864687,
 -0.018899911809224856,
 0.0012131269190881972,
 -0.013518155405549754,
 0.004106458697548069,
 0.013683944791868354,
 -0.001726434314489366,
 0.002674936986331728,
 -0.019397279036858086,
 0.009411697173662968,
 -0.014742441315190412,
 0.03341279887607338,
 -0.00022098517852979683,
 0.01270196479912265,
 -0.00836595325021654,
 -0.0056176864862935195,
 0.001176462146707871,
 0.0037206811426087954,
 -0.012568059240799541,
 -0.003135638277431428,
 0.005646380534505615,
 0.018644852361131704,
 -0.01050845335925705,
 0.002842319865558178,
 -0.03387190737275715,
 0.01217271653746162,
 -0.025850284563730876,
 0.012083445544364505,
 -0.010814524138175291,
 -0.006548653751230285,
 -0.00920764905639491,
 0.017433319051366677,
 -0.011662597641275321,
 0.021756578000397157,
 0.009430826539137698,
 0.0238863264408846,
 -0.0017025224912306331,
 -0.00260798374150

In [99]:


# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY')
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = pinecone.Pinecone(api_key=api_key)

In [32]:
from pinecone import ServerlessSpec, PodSpec

use_serverless = os.environ.get("USE_SERVERLESS", "False").lower() == "true"
if use_serverless:
    spec = ServerlessSpec(cloud='aws', region='us-west-2')
else:
    spec = PodSpec(environment=environment)

In [98]:
index_name = 'ddm-eta'

In [35]:
use_serverless

False

In [42]:
api_key

'15360e69-25ba-4752-b703-c9b999504dba'

In [100]:
index = pc.Index(index_name)
# wait a moment for connection
time.sleep(1)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [114]:
documents

[Document(page_content='GOVERNMENT OF INDIA\nBUDGET 2023-2024\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2023', metadata={'source': 'documents/budget_speech.pdf', 'page': 0}),
 Document(page_content='CONTENTS \nPART-A \n Page No.  \n\uf0b7 Introduction 1 \n\uf0b7 Achievements since 2014: Leaving no one behind 2 \n\uf0b7 Vision for Amrit Kaal  – an empowered and inclusive economy 3 \n\uf0b7 Priorities of this Budget 5 \ni. Inclusive Development  \nii. Reaching the Last Mile \niii. Infrastructure and Investment \niv. Unleashing the Potential \nv. Green Growth \nvi. Youth Power  \nvii. Financial Sector  \n \n \n \n \n \n \n \n \n\uf0b7 Fiscal Management 24 \nPART B  \n  \nIndirect Taxes  27 \n\uf0b7 Green Mobility  \n\uf0b7 Electronics   \n\uf0b7 Electrical   \n\uf0b7 Chemicals and Petrochemicals   \n\uf0b7 Marine products  \n\uf0b7 Lab Grown Diamonds  \n\uf0b7 Precious Metals  \n\uf0b7 Metals  \n\uf0b7 Compounded Rubber  \n\uf0b7 Cigarettes  \n  \nDirect Taxes  30

In [101]:
from tqdm.auto import tqdm  # for progress bar
import uuid

batch_size = 100

data = documents
for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data[i:i_end]
    # generate unique ids for each chunk
    ids = [str(uuid.uuid4()) for _ in batch]
    # get text to embed
    texts = [x.page_content for x in batch]
    # embed text
    embeds = embeddings.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x.page_content,
         'source': x.metadata['source'],
         'page': x.metadata['page']} for x in batch
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

100%|██████████| 2/2 [00:05<00:00,  2.77s/it]


In [102]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00141,
 'namespaces': {'': {'vector_count': 141}},
 'total_vector_count': 141}

In [103]:
text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = PineconeVectorStore(
    index, embeddings, text_field
)

In [104]:


query = "How much the agriculture target will be increased by how many crore?"

vectorstore.similarity_search(query, k=3)

[Document(page_content="7 \n \n \n farmers in contributing to the health of fellow citizens by growing these \n‘Shree Anna’.   \n22. Now to make India a global hub for ' Shree Anna' , the Indian Institute \nof Millet Research, Hyderabad  will be supported as the Centre of Excellence \nfor sharing best practices, research and technologies at the international \nlevel.    \nAgriculture Credit  \n23. The agriculture credit target will be increased  \nto ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  \nFisheries \n24. We will launch a new sub-scheme of PM Matsya Sampada Yojana \nwith targeted investment of ` 6,000 crore to further enable activities of \nfishermen, fish vendors, and micro & small enterprises, improve value chain \nefficiencies, and expand the market. \nCooperation", metadata={'page': 10.0, 'source': 'documents/budget_speech.pdf'}),
 Document(page_content='improved access to education, health and nutrition, road and telecom \nconnectivity, and sustaina

In [105]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt
    

In [106]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    7 
 
 
 farmers in contributing to the health of fellow citizens by growing these 
‘Shree Anna’.   
22. Now to make India a global hub for ' Shree Anna' , the Indian Institute 
of Millet Research, Hyderabad  will be supported as the Centre of Excellence 
for sharing best practices, research and technologies at the international 
level.    
Agriculture Credit  
23. The agriculture credit target will be increased  
to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.  
Fisheries 
24. We will launch a new sub-scheme of PM Matsya Sampada Yojana 
with targeted investment of ` 6,000 crore to further enable activities of 
fishermen, fish vendors, and micro & small enterprises, improve value chain 
efficiencies, and expand the market. 
Cooperation
improved access to education, health and nutrition, road and telecom 
connectivity, and sustainable livelihood opportunities. An amount  
of ` 15,000 crore will be made

In [107]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [108]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand Finance theory.")
]

In [109]:
from langchain_community.chat_models import ChatOpenAI
chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

In [111]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

The agriculture credit target will be increased to ₹ 20 lakh crore.


In [86]:
prompt = HumanMessage(
    content=augment_prompt("Who is the Chief Minister of Telangana?")
)
# add to messages
messages.append(prompt)

res = chat.invoke(messages)

print(res.content)

I'm sorry, but the context provided does not mention the Chief Minister of Telangana. Would you like me to try to answer a different question or provide information on a different topic?
